Кобзарь О.С. Хабибуллин Р.А. 2019 г.

# Инклинометрия и конструкция скважин

Уточнить параметры!!!

Геометрия профиля ствола скважины значительно влияет на процесс добычи. При горизонтальной конструкции скважины дебит ее как правило увеличивается, за счет большей площади контакта с пластом.

На процесс расчета модели скважины, в частности для получения КРД и КРТ, необходимо знать абсолютную глубину при различных глубинах вдоль ствола скважины и угол наклона рассматриваемого участка.

Кроме того, профиль ствола скважины накладывает свои ограничения на установку погружного оборудования. Процесс добычи с помощью УШГН в наклонно-направленных скважинах может быть значительно осложнен или невозможен. При применении УЭЦН необходимо учитывать два фактора:

* ЭЦН должен быть размещен на неискривлённом участке (slant section) с темпом набора кривизны не более 2 минут на 10 м. В противном случае возникающие при работе насоса вибрации и напряжения на валу быстро выведут его из строя. При строительстве скважин данный участок делают длиной порядка 100 м.

* При самом спуске ЭЦН до предполагаемой глубины размещения не должно быть участков с темпом набора кривизны более 2 градусов на 10 м. Иначе при установке оборудования произойдут необратимые деформации, которые приведут к скорому отказу.

При отсутствии инклинометрии можно воспользоваться простой моделью конструкции скважины. По известным точкам - данным по глубине и удлинению в точке - проинтерполировать значения и получить примерный профиль скважины. Исходными данными будут являться глубины и удлинения: кондуктора (по умолчанию 500 м - вертикальный), спуска оборудования, забоя скважины.

In [1]:
import sys
sys.path.append('../')

import uniflocpy.uWell.deviation_survey as dev_sev
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uWell.uPipe as Pipe
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt

## Профиль реальной скважины по инклинометрии

In [2]:
path_to_file = './data/test_deviation_survey_data.xls'

wds = dev_sev.well_deviation_survey()

wds.load_deviation_survey(path_to_file)

wds.calc_all()

In [3]:
x = wds.deviation_survey_dataframe['Координата Х (инклинометрия)']
y = wds.deviation_survey_dataframe['Координата Y (инклинометрия)']
z = wds.deviation_survey_dataframe['Вертикальная отметка'] * (-1)  # перевод в альтитуду
trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    marker=dict(
        size=4,
        color=z,
        colorscale='Viridis',
    )
    )

data = [trace1]


layout = dict(title = '3D профиль скважины',  yaxis=dict(autorange='reversed'))
fig = dict(data=data, layout=layout)
iplot(fig, filename='simple-3d-scatter')

In [4]:
def trace(data_x, data_y, namexy):
    tracep = go.Scattergl(
        x = data_x,
        y = data_y,
        name = namexy,
        mode = 'lines'
    )
    return tracep

def plot():
    layout = dict(title = 'Параметры скважины вдоль ствола',  yaxis=dict(title = 'Кривизна, град/10м'), 
                  xaxis=dict(title = 'Глубина скважины, м'))

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')
def plot_along_depth(this_title):
    layout = dict(title = this_title,  yaxis=dict(title = 'Измеренная глубина, м', autorange='reversed'))

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

In [5]:
h_vert_m = wds.column_h_mes_m
trace1 = trace(h_vert_m, wds.curvature_rate_interpolate_func(h_vert_m),'Кривизна, град/10м')
data=[trace1]
plot()

In [6]:
h_vert_m = wds.column_h_mes_m
trace1 = trace(wds.get_vert_angle_grad(h_vert_m),h_vert_m,'Измеренная глубина, м')
data=[trace1]
plot_along_depth("Угол наклона от вертикали, град")

# Простой профиль скважины по точкам

In [7]:
simple_dev_sev = dev_sev.simple_well_deviation_survey()
simple_dev_sev.calc_all()
x_displacement_init_data = simple_dev_sev.interpolation_x_displacement_by_h_mes(
    simple_dev_sev.h_mes_init_data_for_interpolation_m)

In [8]:
trace1 = go.Scatter3d(
    x=simple_dev_sev.x_displacement_m,
    y=simple_dev_sev.y_displacement_m,
    z=simple_dev_sev.h_vert_m * (-1),  # перевод в альтитуду,
    marker=dict(
        size=4,
        color=z,
        colorscale='Viridis',
    ),
    name = 'Построенный профиль',
    )

trace2 = go.Scatter3d(
    x=x_displacement_init_data,
    y=x_displacement_init_data * 0,
    z=simple_dev_sev.h_vert_init_data_for_interpolation_m * (-1),
    mode = 'markers',
    marker = dict(
        size=10
    ),
    name = 'Исходные точки',
    )

data = [trace1, trace2]


layout = dict(title = '3D профиль простой скважины',  yaxis=dict(autorange='reversed'))
fig = dict(data=data, layout=layout)
iplot(fig, filename='simple-3d-scatter')

In [9]:
h_vert_m = simple_dev_sev.h_mes_m
trace1 = trace(h_vert_m, simple_dev_sev.get_curvature_rate_grad10m(h_vert_m),'Кривизна, град/10м')
data=[trace1]
plot()

In [10]:
h_vert_m = simple_dev_sev.h_mes_m
trace1 = trace(simple_dev_sev.get_angle_to_horizontal_grad(h_vert_m),h_vert_m,'Измеренная глубина, м')
data=[trace1]
plot_along_depth("Угол наклона от горизонтали, град")